Spherically symmetric stellar wind
---

Consider a simple spherically symmetric stellar wind model.
We use numpy and astropy to conveniently define the model parameters.

In [1]:
import numpy as np
from astropy import units, constants



The model box is given by the radial coordinate $r \in [r_{\star}, r_{\text{out}}] = [1, 10^{4}] \ \text{au}$. The model is discretised on a logarithmically-spaced grid consisting of 1024 elements with $r \in [r_{\text{in}}, r_{\text{out}}] = [10^{-1}, 10^{4}] \ \text{au}$.
Note that $r_{\text{in}} < r_{\star}$, such that several rays hit the stellar surface, since, for concenience, we use the same discretisation for the impact parameters of the rays. We impose a boundary condition at $r=r_{\star}$, such that the part of the model inside the star ($r<r_{\star}$) does not contribute to the resulting observation, and thus cannot be reconstructed.

In [2]:
n_elements = 1024

r_in   = (1.0e-1 * units.au).si.value
r_out  = (1.0e+4 * units.au).si.value
r_star = (1.0e+0 * units.au).si.value

rs = np.logspace(np.log10(r_in), np.log10(r_out), n_elements, dtype=np.float64)

For the velocity field, we assume a typical radially outward directed $\beta$-law,
\begin{equation*}
    v(r) \ = \ v_{\star} \ + \ \left( v_{\infty} - v_{\star} \right) \left(1 - \frac{r_{\star}}{r}\right)^{\beta} ,
\end{equation*}
in which $v_{0} = 0.1 \ \text{km}/ \text{s}$,  $v_{\infty} = 20 \ \text{km}/ \text{s}$, and $\beta=0.5$.

In [3]:
v_in  = (1.0e-1 * units.km / units.s).si.value
v_inf = (2.0e+1 * units.km / units.s).si.value
beta  = 0.5

v = np.empty_like(rs)
v[rs <= r_star] = 0.01
v[rs >  r_star] = v_in + (v_inf - v_in) * (1.0 - r_star / rs[rs > r_star])**beta

We assume the density and velocity to be related through the conservation of mass, such that,
\begin{equation*}
    \rho \left( r \right) \ = \ \frac{\dot{M}}{4 \pi r^{2} \, v(r)},
\end{equation*}
where, for the mass-loss rate, we take a typical value of $\dot{M} = 5.0 \times 10^{-6} \ M_{\odot} / \text{yr}$.

In [4]:
Mdot = (3.0e-6 * units.M_sun / units.yr).si.value

rho  = Mdot / (4.0 * np.pi * rs**2 * v)

The CO abundance is assumed to be proportional to the density, such that, $n^{\text{CO}}(r) = 3.0 \times 10^{-4} \, N_{A} \, \rho(r) / m^{\text{H}_2}$, with $N_{A}$ Avogadro's number, and $m^{\text{H}_2} = 2.02 \ \text{g}/\text{mol}$, the molar mass of $\text{H}_{2}$.

In [5]:
n_CO             = (3.0e-4 * constants.N_A.si.value / 2.02e-3) * rho
n_CO[rs<=r_star] = n_CO[n_CO<np.inf].max() # Set to max inside star

For the gas temperature, we assume a power law,
\begin{equation*}
    T(r) \ = \ T_{\star} \left(\frac{r_{\star}}{r}\right)^{\epsilon} ,
\end{equation*}
with $T_{\star} = 2500 \ \text{K}$, and $\epsilon=0.6$.

In [6]:
T_star = (2.5e+3 * units.K).si.value
epsilon = 0.6

T = np.empty_like(rs)
T[rs <= r_star] = T_star
T[rs >  r_star] = T_star * (r_star / rs[rs > r_star])**epsilon

Finally, we assume a constant turbulent velocity $v_{\text{turb}}(r) = 1 \ \text{km}/\text{s}$.

In [7]:
v_turb = (1.0e+0 * units.km / units.s).si.value

We base our reconstructions on synthetic observations of two commonly observed rotational CO lines $J = \{(3-2), \, (7-6)\}$, which we observe, each in 50 frequency bins, centred around the lines, with a spacing of 1.02 km/s.

In [8]:
from pomme.lines import Line

lines = [Line('CO', i) for i in [2, 6]]

You have selected line:
    CO(J=3-2)
Please check the properties that were inferred:
    Frequency         3.457959899e+11  Hz
    Einstein A coeff  2.497000000e-06  1/s
    Molar mass        28.0101          g/mol
You have selected line:
    CO(J=7-6)
Please check the properties that were inferred:
    Frequency         8.066518060e+11  Hz
    Einstein A coeff  3.422000000e-05  1/s
    Molar mass        28.0101          g/mol


/home/frederikd/.local/lib/python3.9/site-packages/astroquery/lamda/core.py:145: UserWarning: The first time a LAMDA function is called, it must assemble a list of valid molecules and URLs.  This list will be cached so future operations will be faster.
  warnings.warn("The first time a LAMDA function is called, it must "


In [10]:
import torch
from pomme.model import TensorModel, SphericalModel
from pomme.utils import planck, T_CMB

model = TensorModel(sizes=r_out, shape=n_elements)

# Define and initialise the model variables
model['log_r'         ] = np.log(rs)
model['log_CO'        ] = np.log(n_CO)
model['log_turbulence'] = np.log(v_turb) * np.ones(n_elements)
model['log_v_in'      ] = np.log(v_in)
model['log_v_inf'     ] = np.log(v_inf)
model['log_beta'      ] = np.log(beta)
model['log_epsilon'   ] = np.log(epsilon)
model['log_T_star'    ] = np.log(T_star)
model['log_r_star'    ] = np.log(r_star)

model.fix_all()
model.save('model_truth.h5')


def get_velocity(model):
    """
    Get the velocity from the TensorModel.
    """
    # Extract parameters
    r      = torch.exp(model['log_r'])
    v_in   = torch.exp(model['log_v_in'])
    v_inf  = torch.exp(model['log_v_inf'])
    beta   = torch.exp(model['log_beta'])
    R_star = torch.exp(model['log_r_star'])
    # Compute velocity
    v = torch.empty_like(r)
    v[r <= r_star] = v_in
    v[r >  r_star] = v_in + (v_inf - v_in) * (1.0 - r_star / r[r > r_star])**beta
    # Return
    return v


def get_temperature(model):
    """
    Get the temperature from the TensorModel.
    """
    # Extract parameters
    r       = torch.exp(model['log_r'])
    T_star  = torch.exp(model['log_T_star'])
    epsilon = torch.exp(model['log_epsilon'])
    r_star  = torch.exp(model['log_r_star'])
    # Compute temperature
    T = torch.empty_like(r)    
    T[r <= r_star] = T_star
    T[r >  r_star] = T_star * (r_star / r[r > r_star])**epsilon
    # Return
    return T


def get_abundance(model):
    """
    Get the abundance from the TensorModel.
    """
    return torch.exp(model['log_CO'])


def get_turbulence(model):
    """
    Get the turbulence from the TensorModel.
    """
    return torch.exp(model['log_turbulence'])


def get_boundary_condition(model, frequency, b):
    """
    Get the boundary condition from the TensorModel.
    """
    # Extract parameters
    T_star = torch.exp(model['log_T_star'])
    r_star = torch.exp(model['log_r_star'])
    # Compute boundary condition
    if b > r_star:
        return planck(temperature=T_CMB, frequency=frequency)
    else:
        return planck(temperature=T_star, frequency=frequency)


smodel = SphericalModel(rs, model, r_star=r_star)
smodel.get_velocity           = get_velocity
smodel.get_abundance          = get_abundance
smodel.get_turbulence         = get_turbulence
smodel.get_temperature        = get_temperature
smodel.get_boundary_condition = get_boundary_condition

Define the velocity/frequency range of interest around the lines.

In [ ]:
# Frequency data
vdiff = 500   # velocity increment size [m/s]
nfreq =  50   # number of frequencies

velocities  = nfreq * vdiff * torch.linspace(-1, +1, nfreq, dtype=torch.float64)
frequencies = [(1.0 + velocities / constants.c.si.value) * line.frequency for line in lines]

In [ ]:
obss = smodel.image(lines, frequencies, r_max=r_out)

In spherical symmetry, the loss that assumes a steady state and enforces the continuity equation, reads,
\begin{equation*}
    \mathcal{L}[\rho, v]
    \ = \
    \int_{0}^{\infty} 4\pi r^{2} \text{d}r \left\{ \frac{1}{\rho \, r^{2}} \, \partial_{r} \left( r^{2} \rho \, v \right) \right\}^{2} .
\end{equation*}

In [11]:
def steady_state_cont_loss(smodel):
    """
    Loss assuming steady state hydrodynamics, i.e. vanishing time derivatives.
    """
    # Get a mask for the elements outsife the star
    outside_star = torch.from_numpy(smodel.rs) > torch.exp(smodel.model_1D['log_r_star'])
    # Get the model variables
    rho = smodel.get_abundance(smodel.model_1D)[outside_star]
    v_r = smodel.get_velocity (smodel.model_1D)[outside_star]
    r   = torch.from_numpy(smodel.rs)          [outside_star]
    # Continuity equation (steady state): div(ρ v) = 0
    loss_cont = smodel.diff_r(r**2 * rho * v_r, r) / (rho*r**2)
    # Compute the mean squared losses
    loss = torch.mean(4.0*torch.pi*r**2*(loss_cont)**2)
    # Return losses
    return loss

[]

In [ ]:
from pomme.loss  import Loss, diff_loss
from torch.optim import Adam


obss_avg = obss.mean(axis=1)
obss_rel = torch.einsum("ij, i -> ij", obss, 1.0 / obss.mean(axis=1))

# Get a mask for the elements outsife the star
outside_star = torch.from_numpy(smodel.rs) > torch.exp(smodel.model_1D['log_R_star'])


def fit(losses, smodel, lines, frequencies, N_epochs=10, lr=1.0e-1, w_avg=1.0, w_rel=1.0, w_reg=1.0, w_cnt=1.0):

    # Define optimiser
    optimizer = Adam(model.free_parameters(), lr=lr)

    for _ in tqdm(range(N_epochs)):
        # Forward model
        imgs     = smodel.image(lines, frequencies, r_max=r_out)
        imgs_avg = imgs.mean(axis=1)
        imgs_rel = torch.einsum("ij, i -> ij", imgs, 1.0 / imgs.mean(axis=1))

        # Compute the reproduction loss
        losses['avg'] = w_avg * torch.nn.functional.mse_loss(imgs_avg, obss_avg)
        losses['rel'] = w_rel * torch.nn.functional.mse_loss(imgs_rel, obss_rel)
        # Compute the regularisation loss
        losses['reg'] = w_reg * (   diff_loss(smodel.model_1D['log_CO'         ][outside_star]) \
                                  + diff_loss(smodel.model_1D['log_velocity'   ][outside_star]) \
                                  + diff_loss(smodel.model_1D['log_temperature'][outside_star]) )
        # Compute the hydrodynamic loss   
        losses['cnt'] = w_cnt * steady_state_cont_loss(smodel)

        # Set gradients to zero
        optimizer.zero_grad()
        # Backpropagate gradients
        losses.tot().backward()
        # Update parameters
        optimizer.step()

    return imgs, losses

In [ ]:
obss = torch.load('obss.pt')

smodel = SphericalModel(
    rs       = smodel_truth.rs,
    model_1D = TensorModel.load('model_truth.h5'),
    r_star   = smodel_truth.r_star,
)
smodel.get_abundance          = get_abundance
smodel.get_velocity           = get_velocity
smodel.get_temperature        = get_temperature
smodel.get_turbulence         = get_turbulence
smodel.get_boundary_condition = get_boundary_condition

log_n_CO_init = np.log(5.0e+14*(smodel.rs.min()/smodel.rs)**2)

smodel.model_1D['log_CO'] = log_n_CO_init.copy()
# smodel.model_1D.free(['log_CO', 'log_v_in', 'log_v_inf', 'log_beta', 'log_T_in', 'log_epsilon'])
smodel.model_1D.free(['log_CO'])

# losses = Loss(['avg', 'rel', 'reg'])
losses = Loss(['avg', 'rel'])




from torch.optim import Adam
from tqdm        import tqdm

        




imgs, losses, a_evol = fit(losses, smodel, lines, frequencies, obss, N_epochs=3, lr=1.0e-1, w_avg=1.0, w_rel=1.0e+0, w_reg=1.0e-0, w_cnt=1.0e+0)
losses.renormalise_all()
losses.reset()

imgs, losses, a_evol = fit(losses, smodel, lines, frequencies, obss, N_epochs=10, lr=1.0e-1, w_avg=1.0, w_rel=1.0e+0, w_reg=1.0e-0, w_cnt=1.0e+0)
losses.plot()
plt.savefig(f'CO/losses_1_no_diff.png')

imgs, losses, a_evol = fit(losses, smodel, lines, frequencies, obss, N_epochs=500, lr=1.0e-1, w_avg=1.0, w_rel=1.0e+0, w_reg=1.0e-0, w_cnt=1.0e+0)
losses.plot()
plt.savefig(f'CO/losses_2_nodiff.png')

torch.save(imgs, f'CO/imgs_nodiff.pt')

smodel.model_1D.save(f'CO/model_nodiff.h5')